# Bitcoin Graph Analysis

In [ ]:
import os
import csv
import sqlite3
import statistics

import igraph
import pandas as pd

from time_manager import get_unixtime

In [ ]:
IGRAPH_PICKLE = ''
g = igraph.Graph.Read_Pickle(IGRAPH_PICKLE)

In [ ]:
partition = g.community_leiden(objective_function='modularity')

df = pd.DataFrame()
df['membership'] = partition.membership

dfcnt = df['membership'].value_counts()
largest_comm = dfcnt.sort_values(ascending=False).index

In [ ]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.execute('''ATTACH DATABASE 'dbv3-index.db' AS DBINDEX;''')
cur.execute('''ATTACH DATABASE 'dbv3-core.db' AS DBCORE;''')
cur.execute('''ATTACH DATABASE 'kisa.db' AS DBCLUSTER;''')
conn.commit()

In [ ]:
QT = '''
SELECT DBINDEX.TxID.txid
FROM DBINDEX.TxID
WHERE DBINDEX.TxID.id = ?;'''

QA = '''
SELECT DBINDEX.AddrID.addr
FROM DBINDEX.AddrID
WHERE DBINDEX.AddrID.id = ?;'''

QIA = '''
SELECT DBINDEX.AddrID.id
FROM DBINDEX.AddrID
WHERE DBINDEX.AddrID.addr = ?;'''

QCNTTX = '''
SELECT COUNT(DISTINCT tx)
FROM (
SELECT TxIn.tx
FROM TxIn
INNER JOIN TxOut ON TxOut.tx = TxIn.ptx AND
                    TxOut.n = TxIn.pn
WHERE TxOut.addr = ?
UNION 
SELECT TxOut.tx
FROM TxOut
WHERE TxOut.addr = ?);'''

QINTX = '''
SELECT COUNT(DISTINCT TxIn.tx)
FROM TxIn
INNER JOIN TxOut ON TxOut.tx = TxIn.ptx AND
                    TxOut.n = TxIn.pn
WHERE TxOut.addr = ?;'''

QOUTTX = '''
SELECT COUNT(DISTINCT TxOut.tx)
FROM TxOut
WHERE TxOut.addr = ?;'''

QINBTC = '''
SELECT SUM(TxOut.btc)
FROM TxIn 
INNER JOIN TxOut ON TxOut.tx = TxIn.ptx AND
                    TxOut.n = TxIn.pn
WHERE TxOut.addr = ?;'''

QOUTBTC = '''
SELECT SUM(TxOut.btc)
FROM TxOut
WHERE TxOut.addr = ?;'''

QBAL = '''
SELECT Income.value-Outcome.value AS Balance
FROM
(SELECT SUM(btc) AS value
 FROM TxOut
 WHERE TxOut.addr = ?) AS Income,
(SELECT SUM(btc) AS value
 FROM TxIn
 INNER JOIN TxOut ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n
 WHERE TxOut.addr = ?) AS Outcome;'''

QDMIN = '''
SELECT MIN(BlkTime.unixtime)
FROM BlkTime
INNER JOIN BlkTx ON BlkTx.blk = BlkTime.blk
WHERE BlkTx.tx IN
(SELECT tx
 FROM (
 SELECT DISTINCT TxIn.tx
 FROM TxIn
 INNER JOIN TxOut ON TxOut.tx = TxIn.ptx AND
                     TxOut.n = TxIn.pn
 WHERE TxOut.addr = ?
 UNION 
 SELECT DISTINCT TxOut.tx
 FROM TxOut
 WHERE TxOut.addr = ?));'''

QDMAX = '''
SELECT MAX(BlkTime.unixtime)
FROM BlkTime
INNER JOIN BlkTx ON BlkTx.blk = BlkTime.blk
WHERE BlkTx.tx IN
(SELECT tx
 FROM (
 SELECT DISTINCT TxIn.tx
 FROM TxIn
 INNER JOIN TxOut ON TxOut.tx = TxIn.ptx AND
                     TxOut.n = TxIn.pn
 WHERE TxOut.addr = ?
 UNION 
 SELECT DISTINCT TxOut.tx
 FROM TxOut
 WHERE TxOut.addr = ?));'''

In [ ]:
MAXRANK = len(dfcnt)

with open('output_graph.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['Rank', 'CommunitySize',
                                           'TxInCount', 'TxOutCount', 'SumInBTC', 'SumOutBTC',
                                           'AvgTxInCount', 'AvgTxOutCount', 'AvgSumInBTC', 'AvgSumOutBTC',
                                           'MedTxInCount', 'MedTxOutCount', 'MedSumInBTC', 'MedSumOutBTC',
                                           'StdTxInCount', 'StdTxOutCount', 'StdSumInBTC', 'StdSumOutBTC',],
                            quoting=csv.QUOTE_MINIMAL,
                            lineterminator=os.linesep)
    writer.writeheader()
    for i in range(0, MAXRANK):
        result = dict()
        rank = i+1
        commsize = dfcnt[largest_comm[i]]
        txincnt = list()
        txoutcnt = list()
        suminbtc = list()
        sumoutbtc = list()
        for idx in df[df['membership'] == largest_comm[i]].index:
            addr = g.vs[idx]['name']
            cur.execute(QIA, (addr,))
            addrid = cur.fetchone()[0]
            cur.execute(QINTX, (addrid,))
            txincnt.append(cur.fetchone()[0])
            cur.execute(QOUTTX, (addrid,))
            txoutcnt.append(cur.fetchone()[0])
            cur.execute(QINBTC, (addrid,))
            suminbtc.append(cur.fetchone()[0])
            cur.execute(QOUTBTC, (addrid,))
            sumoutbtc.append(cur.fetchone()[0])
        writer.writerow({'Rank': rank, 
                         'CommunitySize': commsize,
                         'TxInCount': sum(txincnt),
                         'TxOutCount': sum(txoutcnt),
                         'SumInBTC': sum(suminbtc),
                         'SumOutBTC': sum(sumoutbtc),
                         'AvgTxInCount': statistics.mean(txincnt),
                         'AvgTxOutCount': statistics.mean(txoutcnt),
                         'AvgSumInBTC': statistics.mean(suminbtc),
                         'AvgSumOutBTC': statistics.mean(sumoutbtc),
                         'MedTxInCount': statistics.median(txincnt),
                         'MedTxOutCount': statistics.median(txoutcnt),
                         'MedSumInBTC': statistics.median(suminbtc),
                         'MedSumOutBTC': statistics.median(sumoutbtc),
                         'StdTxInCount': statistics.stdev(txincnt),
                         'StdTxOutCount': statistics.stdev(txoutcnt),
                         'StdSumInBTC': statistics.stdev(suminbtc),
                         'StdSumOutBTC': statistics.stdev(sumoutbtc),
                        })
        print(f'{i} Done', end='\r')